# Here we want to curate a dataset of trees

This is challenging because NO PAPERS PROVIDE THEIR DATA

In [2]:
# Read in Tallo and see if any sites have high proportoin of certain trees

import pandas as pd 

df = pd.read_csv('../data/tallo/Tallo.csv')
df

,tree_id,division,family,genus,species,latitude,longitude,stem_diameter_cm,height_m,crown_radius_m,height_outlier,crown_radius_outlier,reference_id
0,T_1,Angiosperm,Betulaceae,Betula,Betula pubescens,67.3950,28.7230,10.0,5.6,1.65,N,N,1
1,T_2,Gymnosperm,Pinaceae,Picea,Picea abies,67.3950,28.7230,10.0,6.6,0.85,N,N,1
2,T_3,Gymnosperm,Pinaceae,Picea,Picea abies,67.3950,28.7230,10.0,8.5,0.75,N,N,1
3,T_4,Gymnosperm,Pinaceae,Picea,Picea abies,67.3950,28.7230,10.0,7.2,0.75,N,N,1
4,T_5,Gymnosperm,Pinaceae,Picea,Picea abies,67.3950,28.7230,10.0,8.1,1.10,N,N,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
498833,T_498834,Angiosperm,Myrtaceae,Eucalyptus,Eucalyptus regnans,-43.1593,146.8434,589.0,79.7,NaN,N,N,69
498834,T_498835,Angiosperm,Myrtaceae,Eucalyptus,Eucalyptus regnans,-43.1213,146.8172,593.0,78.0,NaN,N,N,69
498835,T_498836,Angiosperm,Myrtaceae,Eucalyptus,Eucalyptus regnans,-41.2273,147.9892,605.0,59.0,NaN,N,N,69
498836,T_498837,Angiosperm,Myrtaceae,Eucalyptus,Eucalyptus regnans,-43.0766,146.7702,608.0,73.0,NaN,N,N,69


In [10]:
# What we want to do is group by lat and long and then look at the species distribution try find places with very few
grped = df.groupby(['latitude', 'longitude'])
location_map = {}

for location, values in grped:
    location_map[location] = {}
    family = {}
    genus = {}
    species = {}
    division = {}
    loc_size = len(values)
    for f in set(values['family'].values):
        family[f] = len(values['family']==f)/loc_size
    for f in set(values['genus'].values):
        genus[f] = len(values['genus']==f)/loc_size
    for f in set(values['species'].values):
        species[f] = len(values['species']==f)/loc_size
    for f in set(values['division'].values):
        division[f] = len(values['division']==f)/loc_size
    location_map[location]['size'] = loc_size
    location_map[location]['division'] = division
    location_map[location]['family'] = family
    location_map[location]['genus'] = genus
    location_map[location]['species'] = species

In [ ]:
# (52306, 9550)

In [17]:
# Calculate the number of places which have different numbers...
homo = []
hetro = []

for loc, location in location_map.items():
    if len(location['species']) == 1:
        homo += [c for c in location['species']] # i.e. add in the species
    else:
        hetro.append(loc)

In [21]:
h_counts = {}
for h in homo:
    if not h_counts.get(h):
        h_counts[h] = 1
    else:
        h_counts[h] += 1

In [24]:
# Cool now print assending order
{k: v for k, v in sorted(h_counts.items(), key=lambda item: -item[1])}

{'Quercus ilex': 9221,
 'Pinus pinaster': 8351,
 'Pinus halepensis': 8132,
 'Pinus sylvestris': 5913,
 'Pinus nigra': 4066,
 'Fagus sylvatica': 2025,
 'Pinus pinea': 1897,
 'Quercus suber': 1836,
 'Quercus faginea': 1578,
 'Pinus radiata': 1516,
 'Eucalyptus globulus': 1493,
 'Quercus robur': 1367,
 'Castanea sativa': 805,
 'Quercus petraea': 614,
 'Pinus uncinata': 602,
 'Eucalyptus camaldulensis': 506,
 'Populus nigra': 390,
 nan: 243,
 'Abies alba': 152,
 'Picea abies': 126,
 'Eucalyptus regnans': 92,
 'Quercus rubra': 88,
 'Quercus canariensis': 78,
 'Pseudotsuga menziesii': 62,
 'Pinus taeda': 54,
 'Pinus wallichiana': 49,
 'Chamaecyparis lawsoniana': 39,
 'Picea mariana': 38,
 'Abies balsamea': 31,
 'Quercus macranthera': 29,
 'Picea smithiana': 24,
 'Abies pinsapo': 22,
 'Platanus orientalis': 21,
 'Liriodendron tulipifera': 20,
 'Cedrus deodara': 20,
 'Eucalyptus baxteri': 18,
 'Fagus orientalis': 18,
 'Tilia cordata': 18,
 'Eucalyptus obliqua': 17,
 'Pinus ponderosa': 17,
 'Fr

## Pick the species with more than 500 samples

1. For species with > 500 samples, we want to pick a tree from that location
2. Download 100m2 from that location (around the centre point)
3. pick pixel/location with NDVI > 0.7 within that 100m2
4. Use that as training
5. Test prediction on sites that were not included in training with the same species

In [28]:
sub_df

,tree_id,division,family,genus,species,latitude,longitude,stem_diameter_cm,height_m,crown_radius_m,height_outlier,crown_radius_outlier,reference_id
6,T_7,Gymnosperm,Pinaceae,Pinus,Pinus sylvestris,67.7380,29.6490,10.0,8.6,0.70,N,N,1
7,T_8,Gymnosperm,Pinaceae,Pinus,Pinus sylvestris,67.7380,29.6490,10.0,6.3,0.80,N,N,1
8,T_9,Gymnosperm,Pinaceae,Pinus,Pinus sylvestris,67.7380,29.6490,10.0,9.0,0.65,N,N,1
10,T_11,Gymnosperm,Pinaceae,Pinus,Pinus sylvestris,67.3950,28.7230,10.1,10.1,0.60,N,N,1
12,T_13,Gymnosperm,Pinaceae,Pinus,Pinus sylvestris,67.3950,28.7230,10.1,9.7,0.85,N,N,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
497864,T_497865,Angiosperm,Fagaceae,Castanea,Castanea sativa,46.3800,6.8500,66.0,11.7,6.00,N,N,67
498775,T_498776,Angiosperm,Myrtaceae,Eucalyptus,Eucalyptus globulus,-42.9639,146.7546,382.0,85.0,NaN,N,N,69
498777,T_498778,Angiosperm,Myrtaceae,Eucalyptus,Eucalyptus globulus,-42.9639,146.7560,388.0,90.7,NaN,N,N,69
498778,T_498779,Angiosperm,Myrtaceae,Eucalyptus,Eucalyptus globulus,-42.9662,146.7574,388.0,85.1,NaN,N,N,69


In [37]:
species_of_interest = [c for c, v in h_counts.items() if v > 500]
sub_df = df[df['species'].isin(species_of_interest)]
sub_df.to_csv('../data/sat_public_tallo.csv', index=False)


In [38]:
# Then also just do the lat and long

mean_df = sub_df.groupby(['latitude', 'longitude']).agg({'tree_id': 'first', 
                                                        'division': 'first',
                                                         'family': 'first',
                                                         'genus': 'first', 
                                                         'species': 'first',
                                                        'stem_diameter_cm': lambda x: x.mean(skipna=True),
                                                         'height_m': lambda x: x.mean(skipna=True),
                                                         'crown_radius_m': lambda x: x.mean(skipna=True)})

mean_df.to_csv('../data/sat_public_tallo_mean.csv')
mean_df

tree_id    division     family       genus   
latitude longitude                                                
-43.2561 146.9190   T_498824  Angiosperm  Myrtaceae  Eucalyptus  \
-42.9662 146.7574   T_498779  Angiosperm  Myrtaceae  Eucalyptus   
-42.9639 146.7546   T_498776  Angiosperm  Myrtaceae  Eucalyptus   
         146.7560   T_498778  Angiosperm  Myrtaceae  Eucalyptus   
-42.8000 147.6000   T_332054  Angiosperm  Myrtaceae  Eucalyptus   
...                      ...         ...        ...         ...   
 62.5000 29.2000    T_126621  Gymnosperm   Pinaceae       Pinus   
 63.5000 41.8400    T_496109  Gymnosperm   Pinaceae       Pinus   
 65.2200 33.1700    T_344679  Gymnosperm   Pinaceae       Pinus   
 67.3950 28.7230        T_11  Gymnosperm   Pinaceae       Pinus   
 67.7380 29.6490         T_7  Gymnosperm   Pinaceae       Pinus   

                                species  stem_diameter_cm   height_m   
latitude longitude                                                     
-43.2561 146.9190   Eucalyptus globulus        554.000000  82.300000  \
-42.9662 146.7574   Eucalyptus globulus        388.000000  85.100000   
-42.9639 146.7546   Eucalyptus globulus        382.000000  85.000000   
         146.7560   Eucalyptus globulus        388.000000  90.700000   
-42.8000 147.6000   Eucalyptus globulus         20.933333  20.500000   
...                                 ...               ...        ...   
 62.5000 29.2000       Pinus sylvestris         19.291852  17.693926   
 63.5000 41.8400       Pinus sylvestris          5.368750   6.912500   
 65.2200 33.1700       Pinus sylvestris         28.000000  16.590909   
 67.3950 28.7230       Pinus sylvestris         30.575536  16.269957   
 67.7380 29.6490       Pinus sylvestris         25.903960  14.941611   

                    crown_radius_m  
latitude longitude                  
-43.2561 146.9190              NaN  
-42.9662 146.7574              NaN  
-42.9639 146.7546              NaN  
         146.7560              NaN  
-42.8000 147.6000              NaN  
...                            ...  
 62.5000 29.2000          2.206399  
 63.5000 41.8400          0.653125  
 65.2200 33.1700               NaN  
 67.3950 28.7230          2.020601  
 67.7380 29.6490          1.741667  

[52741 rows x 8 columns]

In [39]:
## Now we want to download these trees and then perform our analysis using planet scope 

In [43]:
# 16 class problem and pick 500 samples from each dataset for training! 
training_df = pd.DataFrame()
for s in species_of_interest:
    species_df = mean_df[mean_df['species'] == s].sample(500)
    training_df = pd.concat([training_df, species_df])
training_df   

,,tree_id,division,family,genus,species,stem_diameter_cm,height_m,crown_radius_m
latitude,longitude,,,,,,,,
43.351,-3.716,T_280016,Angiosperm,Myrtaceae,Eucalyptus,Eucalyptus globulus,33.6,18.0,3.00
43.240,-4.010,T_222717,Angiosperm,Myrtaceae,Eucalyptus,Eucalyptus globulus,12.8,17.0,1.00
43.365,-7.186,T_215807,Angiosperm,Myrtaceae,Eucalyptus,Eucalyptus globulus,9.1,10.0,1.00
43.400,-8.198,T_271436,Angiosperm,Myrtaceae,Eucalyptus,Eucalyptus globulus,28.4,32.0,1.50
37.930,-6.952,T_225814,Angiosperm,Myrtaceae,Eucalyptus,Eucalyptus globulus,13.4,11.0,1.00
...,...,...,...,...,...,...,...,...,...
42.711,-8.255,T_285254,Angiosperm,Fagaceae,Quercus,Quercus robur,38.7,7.5,2.00
43.205,-3.948,T_234319,Angiosperm,Fagaceae,Quercus,Quercus robur,15.1,6.0,2.50
42.929,-8.645,T_300350,Angiosperm,Fagaceae,Quercus,Quercus robur,66.8,15.0,6.50


In [44]:
# 8000 training samples
training_df.to_csv('../data/training_public_tallo.csv')

In [45]:
training_df.division.value_counts()

division
Angiosperm    4500
Gymnosperm    3500
Name: count, dtype: int64

In [46]:
training_df.family.value_counts()

family
Pinaceae     3500
Fagaceae     3500
Myrtaceae    1000
Name: count, dtype: int64

In [47]:
training_df.genus.value_counts()

genus
Pinus         3500
Quercus       2500
Eucalyptus    1000
Castanea       500
Fagus          500
Name: count, dtype: int64

In [48]:
training_df.species.value_counts()

species
Eucalyptus globulus         500
Eucalyptus camaldulensis    500
Pinus radiata               500
Quercus ilex                500
Pinus halepensis            500
Quercus suber               500
Pinus pinea                 500
Quercus faginea             500
Pinus pinaster              500
Castanea sativa             500
Pinus nigra                 500
Pinus sylvestris            500
Pinus uncinata              500
Quercus petraea             500
Fagus sylvatica             500
Quercus robur               500
Name: count, dtype: int64